### Download VOC 2007 and save it to locally

In [5]:
import os

root_dir = '/content/data'
data_exist = False

try:
  os.mkdir(root_dir) 
  os.chdir(root_dir) 
except:
  data_exist = True

In [ ]:
if not data_exist:
  !curl -LO "http://host.robots.ox.ac.uk/pascal/VOC/voc2012/VOCtrainval_11-May-2012.tar"
  !curl -LO "http://host.robots.ox.ac.uk/pascal/VOC/voc2007/VOCtrainval_06-Nov-2007.tar"
  !curl -LO "http://host.robots.ox.ac.uk/pascal/VOC/voc2007/VOCtest_06-Nov-2007.tar"
  !tar -xvf "VOCtrainval_06-Nov-2007.tar"
  !tar -xvf "VOCtest_06-Nov-2007.tar"
  !tar -xvf "VOCtrainval_11-May-2012.tar"
  !rm "VOCtrainval_06-Nov-2007.tar"
  !rm "VOCtest_06-Nov-2007.tar"
  !rm "VOCtrainval_11-May-2012.tar"  


In [1]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
from torchvision import models
from torch.autograd import Variable

from net import vgg16, vgg16_bn
from resnet_yolo import resnet50, resnet18
from yoloLoss import yoloLoss
from dataset import yoloDataset

from visualizer import Visualizer
import time
import numpy as np

In [6]:
file_root = root_dir +  '/VOCdevkit/VOC2007/JPEGImages/'
project_path = '/content/gdrive/My Drive/Colab Notebooks/pytorch-YOLO-v1'
learning_rate = 0.001
num_epochs = 50
batch_size = 24
use_resnet = True
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
if use_resnet:
    net = resnet50()
else:
    net = vgg16_bn()


In [ ]:
print('load pre-trined model')
if use_resnet:
    resnet = models.resnet50(pretrained=True)
    new_state_dict = resnet.state_dict()
    dd = net.state_dict()
    for k in new_state_dict.keys():
        print(k)
        if k in dd.keys() and not k.startswith('fc'):
            print('yes')
            dd[k] = new_state_dict[k]
    net.load_state_dict(dd)
else:
    vgg = models.vgg16_bn(pretrained=True)
    new_state_dict = vgg.state_dict()
    dd = net.state_dict()
    for k in new_state_dict.keys():
        print(k)
        if k in dd.keys() and k.startswith('features'):
            print('yes')
            dd[k] = new_state_dict[k]
    net.load_state_dict(dd)


In [0]:
criterion = yoloLoss(7,2,5,0.5)

net = net.to(device)
net.train()
# different learning rate
params=[]
params_dict = dict(net.named_parameters())
for key,value in params_dict.items():
    if key.startswith('features'):
        params += [{'params':[value],'lr':learning_rate*1}]
    else:
        params += [{'params':[value],'lr':learning_rate}]
optimizer = torch.optim.SGD(params, lr=learning_rate, momentum=0.9, weight_decay=5e-4)

### Data loading

In [ ]:
train_dataset = yoloDataset(root=file_root,list_file=[os.path.join(project_path, 'voc2007.txt'), os.path.join(project_path, 'voc2012.txt')],train=True,transform = [transforms.ToTensor()] )
train_loader = DataLoader(train_dataset,batch_size=batch_size,shuffle=True,num_workers=4)
# test_dataset = yoloDataset(root=file_root,list_file='voc07_test.txt',train=False,transform = [transforms.ToTensor()] )
test_dataset = yoloDataset(root=file_root,list_file=os.path.join(project_path, 'voc2007test.txt'),train=False,transform = [transforms.ToTensor()] )
test_loader = DataLoader(test_dataset,batch_size=batch_size,shuffle=False,num_workers=4)
print('the dataset has %d images' % (len(train_dataset)))
print('the batch_size is %d' % (batch_size))

### Train loop

In [ ]:
num_iter = 0
vis = Visualizer()
best_test_loss = np.inf

for epoch in range(num_epochs):
    net.train()
    if epoch == 1:
        learning_rate = 0.0005
    if epoch == 2:
        learning_rate = 0.00075
    if epoch == 3:
        learning_rate = 0.001
    if epoch == 30:
        learning_rate=0.0001
    if epoch == 40:
        learning_rate=0.00001

    for param_group in optimizer.param_groups:
        param_group['lr'] = learning_rate
    
    print('\n\nStarting epoch %d / %d' % (epoch + 1, num_epochs))
    print('Learning Rate for this epoch: {}'.format(learning_rate))
    
    total_loss = 0.
    
    for i,(images,target) in enumerate(train_loader):
        images = images.to(device)
        target = target.to(device)
        pred = net(images)
        loss = criterion(pred,target)
        total_loss += loss.item()
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if (i+1) % 5 == 0:
            print ('Epoch [%d/%d], Iter [%d/%d] Loss: %.4f, average_loss: %.4f' 
            %(epoch+1, num_epochs, i+1, len(train_loader), loss.item(), total_loss / (i+1)))
            num_iter += 1
            vis.add_log(epoch, ("loss_train", total_loss/(i+1)))

    #validation
    validation_loss = 0.0
    net.eval()
    for i,(images,target) in enumerate(test_loader):
        images = images.to(device)
        target = target.to(device)     
        with torch.set_grad_enabled(False):
          pred = net(images)
          loss = criterion(pred,target)
          validation_loss += loss.item()
    validation_loss /= len(test_loader)
    vis.add_log(epoch, ("loss_val", validation_loss))
    vis.save(os.path.join(project_path, 'temp.csv'))
    if best_test_loss > validation_loss:
        best_test_loss = validation_loss
        print('get best test loss %.5f' % best_test_loss)
        torch.save(net.state_dict(),'best.pth')
        

    vis.save('train_log{}.csv'.format(time.time()))
    torch.save(net.state_dict(),os.path.join(project_path, 'yolo.pth'))
    vis.plot()

